# Lyrics to DataFrame Conversion
This notebook contains code that takes in our master "all_lyrics_cleaned.json" file and converts it into a DataFrame. We also do some data processing, including updating length to be a duration in seconds, as well as calculating song complexity. This DataFrame is then used later in the project for visualizations.

In [ ]:
import json
import pandas as pd

# calculate the song length in seconds from format "1:45"
def length_in_seconds(length):
    if length == '0':
        return 180
    else:
        colon = length.find(':')
        mins = length[0:colon]
        seconds = length[colon+1:]
        return (int(mins) * 60 + int(seconds))

In [ ]:
with open('all_lyrics_cleaned.json', 'r') as file:
    lyric_data = json.load(file)

df = pd.DataFrame()
lst = []

# account for missing data in genre, append the song to the list so it can
# be concatenated to the dataframe
for year in range(1960, 2019):
    for song in lyric_data[str(year)]:
        song['year'] = year
        if not song.get('genre'):
            song['genre'] = 'Other'
        if song.get('length'):
            song['length'] = length_in_seconds(song['length'])

        # calculate the complexity score here, a linear combination
        complexity = (song['num_verses'] + song['num_unique_words'] + song['density']) - (song['similar_verses_score'] + 1)
        song['complexity'] = complexity

        line = pd.DataFrame.from_records([song])
        lst.append(line)

df = pd.concat(lst, ignore_index=True)

In [ ]:
# all the numerical attributes must be the same type for graphing,
# so we ensure they are all float64
df['length'] = df['length'].astype('float64')
df['num_verses'] = df['num_verses'].astype('float64')
df['num_words'] = df['num_words'].astype('float64')
df['year'] = df['year'].astype('float64')
df['num_unique_words'] = df['num_unique_words'].astype('float64')
df['density'] = df['density'].astype('float64')
df['similar_verses_score'] = df['similar_verses_score'].astype('float64')
df['complexity'] = df['complexity'].astype('float64')

# store the dataframe as a json so we can just call it in our plotting ipynb files
df_json = df.to_json('lyrics_dataframe.json')